In [1]:
%cd ..

/home/dimits/Documents/research/llm_moderation/synthetic_moderation_experiments


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from tasks import preprocessing
from annotation_agreement import aposteriori


CONVERSATION_DIR = "data/discussions_output"
ANNOTATION_DIR = "data/annotations_output" 

In [3]:
conv_df = preprocessing.import_and_format_conversations(CONVERSATION_DIR)
conv_df = conv_df.rename(
    {
        "age": "user_age",
        "ethnicity": "user_ethnicity",
        "is_heterosexual": "user_is_heterosexual",
        "profession": "user_profession",
        "gender": "user_gender",
        "education": "user_education",
        "intent": "user_intent"
    },
    axis=1,
)
conv_df = conv_df[
    [
        "id",
        "user",
        "moderator_exists",
        "message",
        "user_age",
        "user_is_heterosexual",
        "user_profession",
        "user_gender",
        "user_intent"
    ]
]
conv_df

,id,user,moderator_exists,message,user_age,user_is_heterosexual,user_profession,user_gender,user_intent
0,4a2d5eda-2a24-4e55-afad-c000373889ff,DarkShadow333,True,User DarkShadow333 posted:\n> I'm so sick of t...,29.0,False,employed,Man,Troll
0,4a2d5eda-2a24-4e55-afad-c000373889ff,moderator,True,User moderator posted:\n@DarkShadow333: I unde...,NaN,True,employed,None,None
0,4a2d5eda-2a24-4e55-afad-c000373889ff,TechGuru88,True,User TechGuru88 posted:\nI completely agree wi...,35.0,True,employed,Man,Helpful
0,4a2d5eda-2a24-4e55-afad-c000373889ff,moderator,True,User moderator posted:\n@DarkShadow333: I unde...,NaN,True,employed,None,None
0,4a2d5eda-2a24-4e55-afad-c000373889ff,WiseOwl71,True,User WiseOwl71 posted:\nI must respectfully di...,71.0,True,employed,Woman,Helpful
...,...,...,...,...,...,...,...,...,...
0,89ade850-1148-4a7a-a2ea-4b0fad9fc54d,moderator,True,"@QueenOfChaos, I understand that this is a sen...",NaN,True,employed,None,None
0,89ade850-1148-4a7a-a2ea-4b0fad9fc54d,DarkShadow333,True,"@QueenOfChaos, you're so right! I'm sick and t...",29.0,False,employed,Man,None
0,89ade850-1148-4a7a-a2ea-4b0fad9fc54d,moderator,True,,NaN,True,employed,None,None
0,89ade850-1148-4a7a-a2ea-4b0fad9fc54d,QueenOfChaos,True,I'm glad someone finally spoke up against thes...,32.0,True,unemployed,Woman,None


In [4]:
annot_df = preprocessing.import_and_format_annotations(
    ANNOTATION_DIR, round=True, sentinel_value=-1
)
annot_df = annot_df.rename(
    {
        "age": "annotator_age",
        "ethnicity": "annotator_ethnicity",
        "is_heterosexual": "annotator_is_heterosexual",
        "profession": "annotator_profession",
        "gender": "annotator_gender",
        "education": "annotator_education",
    },
    axis=1,
)
annot_df
annot_df = annot_df[
    [
        "conv_id",
        "message",
        "toxicity",
        "annotator_age",
        #"annotator_ethnicity",
        "annotator_is_heterosexual",
        "annotator_profession",
        "annotator_gender",
    ]
]
annot_df

,conv_id,message,toxicity,annotator_age,annotator_is_heterosexual,annotator_profession,annotator_gender
0,8bf86b41-6fce-4838-bbc9-24fba2b39e62,I completely agree with @GoldenYears66 that it...,1,30,True,employed,Man
0,8bf86b41-6fce-4838-bbc9-24fba2b39e62,"I completely agree with you, @WiseOwl71. As a ...",1,30,True,employed,Man
0,8bf86b41-6fce-4838-bbc9-24fba2b39e62,"I couldn't agree more with your sentiments, @G...",1,30,True,employed,Man
0,8bf86b41-6fce-4838-bbc9-24fba2b39e62,"I'm not sure what's more disturbing, the fact ...",5,30,True,employed,Man
0,c7134932-480f-43b8-85be-abdeea9a3135,"""Are you kidding me? This is a free country, r...",4,30,False,employed,Woman
...,...,...,...,...,...,...,...
0,75651ec0-7227-438f-963c-3dd0739d6377,"User WiseOwl71 posted:\n**Comment:** ""I unders...",4,70,True,employed,Man
0,75651ec0-7227-438f-963c-3dd0739d6377,"User doing_my_best24 posted:\n**Comment:** ""I ...",4,70,True,employed,Man
0,75651ec0-7227-438f-963c-3dd0739d6377,"User GoldenYears66 posted:\n**Comment:** ""I'm ...",4,70,True,employed,Man
0,75651ec0-7227-438f-963c-3dd0739d6377,"User QueenOfChaos posted:\n**Comment:** ""Oh, l...",4,70,True,employed,Man


In [5]:
combined_df = conv_df.merge(annot_df, left_on=["id", "message"], right_on=["conv_id", "message"])
combined_df.user_age = combined_df.user_age.astype("Int64")
combined_df = combined_df.drop("conv_id", axis=1)
combined_df.head()

,id,user,moderator_exists,message,user_age,user_is_heterosexual,user_profession,user_gender,user_intent,toxicity,annotator_age,annotator_is_heterosexual,annotator_profession,annotator_gender
0,4a2d5eda-2a24-4e55-afad-c000373889ff,DarkShadow333,True,User DarkShadow333 posted:\n> I'm so sick of t...,29,False,employed,Man,Troll,4,30,True,unemployed,Woman
1,4a2d5eda-2a24-4e55-afad-c000373889ff,WiseOwl71,True,User WiseOwl71 posted:\nI must respectfully di...,71,True,employed,Woman,Helpful,1,30,False,employed,Man
2,4a2d5eda-2a24-4e55-afad-c000373889ff,WiseOwl71,True,User WiseOwl71 posted:\nI must respectfully di...,71,True,employed,Woman,Helpful,4,30,True,unemployed,Woman
3,4a2d5eda-2a24-4e55-afad-c000373889ff,WiseOwl71,True,User WiseOwl71 posted:\nI must respectfully di...,71,True,employed,Woman,Helpful,1,70,True,employed,Woman
4,4a2d5eda-2a24-4e55-afad-c000373889ff,QueenOfChaos,True,User QueenOfChaos posted:\nI'm so sick of thes...,32,True,unemployed,Woman,Troll,4,30,False,employed,Man


## Annotator analysis

### Gender

In [6]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3072 entries, 0 to 3071
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         3072 non-null   object
 1   user                       3072 non-null   object
 2   moderator_exists           3072 non-null   bool  
 3   message                    3072 non-null   object
 4   user_age                   3059 non-null   Int64 
 5   user_is_heterosexual       3072 non-null   bool  
 6   user_profession            3072 non-null   object
 7   user_gender                3059 non-null   object
 8   user_intent                2364 non-null   object
 9   toxicity                   3072 non-null   int64 
 10  annotator_age              3072 non-null   int64 
 11  annotator_is_heterosexual  3072 non-null   bool  
 12  annotator_profession       3072 non-null   object
 13  annotator_gender           3072 non-null   object
dtypes: Int64

In [7]:
def group_toxicity_to_list(df, group_by_column):
    """
    Groups 'toxicity' annotations by 'id' and the specified column.
    Returns a list of rows, where each row contains:
    [id, {factor_1: [toxicity_values], factor_2: [toxicity_values], ...}]
    
    Parameters:
    - df (pd.DataFrame): The input dataframe.
    - group_by_column (str): The column name to group by along with 'id'.
    
    Returns:
    - grouped_list (list): A list where each element is [id, {factor: [toxicity_values]}].
    """
    if group_by_column not in df.columns:
        raise ValueError(f"The column '{group_by_column}' is not in the dataframe.")
    
    grouped_list = []
    
    # Group by 'id' and the specified column
    for id_value, group in df.groupby('id'):
        # Create a dictionary mapping each factor to its list of toxicity values
        toxicity_by_factor = (
            group.groupby(group_by_column)['toxicity']
            .apply(list)
            .to_dict()
        )
        
        # Append the [id, toxicity_by_factor] to the result list
        grouped_list.append(toxicity_by_factor)
    
    return grouped_list

In [8]:
def polarization_explained_by_feature(df: pd.DataFrame, feature: str) -> float:
    gender_grouped_toxicity = group_toxicity_to_list(df, feature)

    results = [
        aposteriori.aposteriori_unimodality(list(discussion.values()))
        for discussion in gender_grouped_toxicity
    ]

    explained_num = sum([1 if x else 0 for x in results])
    total_num = len(results)
    assert total_num > 0, "Annotation list can not be empty"

    return explained_num / total_num


In [9]:
polarization_explained_by_feature(combined_df, "annotator_gender")

0.5113636363636364

In [10]:
polarization_explained_by_feature(combined_df, "annotator_is_heterosexual")

0.5113636363636364

In [11]:
polarization_explained_by_feature(combined_df, "annotator_age")

0.42045454545454547